In [1]:
## INPUT
name="Chat Log for Descent Into Fuckery"

In [2]:
players=dict(
{'Connor':'Kallistē',
 'Connor S.':'Kallistē',
 'Lui':'Smallfolk',
 'Julie':'Bethrynne',
 'Kallistē Phlegethina':'Kallistē',
 'Kallistē':'Kallistē',
 'Smallfolk':'Smallfolk',
 'Bethrynne Mistwinter':'Bethrynne',
 'Bethrynne':'Bethrynne'
}
)
plyrs=["Bethrynne","Smallfolk","Kallistē"]

In [ ]:
players=dict(
{'Connor':'Parun',
 'Gruumsh':'Gruumsh',
 'Imelda':'DM',
 'Julie':'Ylva',
 'Lui W.':'Gruumsh',
 'Luise G.':'Gruumsh',
 'Mike G. (GM)':'DM',
 'Parun':'Parun',
 'Ylva Tritina':'Ylva',
 'Trinity':'Trinity',
'Lui, The Unwise Goat':'Gruumsh',
'Lui, They see me misty steppin\'':'Gruumsh',
'Lui':'Gruumsh',
 'Zim':'Zim',
 'Zam':'Zam',
 'Snek':'Snek'
}
)
plyrs=["Parun","Gruumsh","Ylva"]

In [ ]:
players=dict(
{'Dasypus Bellow.':'Dasypus',
 'Edmund':'Nepelope',
 'Eljas O.':'Kääpiöinën',
 'Goat':'Wise Goat',
 'Hak-Bina':'Hak-Bina',
 'Korth':'Korth',
 'Kääpiöinën':'DM',
 'Lui':'Nura',
 'Mike W.':'Korth',
'Nepelope':'Nepelope',
'Nura':'Nura',
 'Roger G. (GM)':'DM',
 'Who is Rea':'Oelita',
 'Zoanne':'Hak-Bina'
}
)
plyrs=["Dasypus","Nepelope","Oelita","Nura","Hak-Bina","Korth","Kääpiöinën","Wise Goat"]

In [14]:
import pandas as pd
import re
from bs4 import BeautifulSoup

skill_labels=['ACROBATICS',
 'ANIMAL HANDLING',
 'ARCANA',
 'ATHLETICS',
 'DECEPTION',
 'HISTORY',
 'INSIGHT',
 'INTIMIDATION',
 'INVESTIGATION',
 'MEDICINE',
 'NATURE',
 'PERCEPTION',
 'PERFORMANCE',
 'PERSUASION',
 'RELIGION',
 'SLEIGHT OF HAND',
 'STEALTH',
 'SURVIVAL']

attr_labels=['STRENGTH',
 'DEXTERITY',
 'CONSTITUTION',
 'INTELLIGENCE',
 'WISDOM',
 'CHARISMA']

save_labels=['STRENGTH SAVE',
 'DEXTERITY SAVE',
 'CONSTITUTION SAVE',
 'INTELLIGENCE SAVE',
 'WISDOM SAVE',
 'CHARISMA SAVE',
 'DEATH SAVE']

print("Loading chat log...")
soup = BeautifulSoup(open('raw/'+name+'.html', 'r').read(), 'html.parser')
msgs=soup.findAll('div',{'class' : re.compile('message \w+')})

print("Parsing messages...")
plyr=None
when=None
df = pd.DataFrame(columns=['Player','Timestamp','Skill/Weapon/Spell','Result', 'Raw','Plus'])
for c,m in enumerate(msgs):
    if c % 500 == 0:
        print(c+1,"/",len(msgs))
    
    ### get player and timestamp
    who=m.findAll('span',{'class': 'by'})
    if who!=[]:
        plyr=who[0].renderContents()
        when=m.find('span',{'class': 'tstamp'}).renderContents()
    

    log=True
    parsing=None
    skchk=m.findAll('div',{'class': ['sheet-rolltemplate-skill','sheet-rolltemplate-simple']})
    atk=m.findAll('div',{'class': ['sheet-rolltemplate-atk']})
    if skchk!=[] and atk!=[]:
        print("Figure out this weirdness! Skill and attack!")
    elif skchk!=[] and atk==[]:
        parsing="Skill"
        mcontent=skchk
    elif atk!=[] and skchk==[]:
        parsing="Attack"
        mcontent=atk

    if parsing!=None:   
        rolls=mcontent[0].find_all('div',{'class': ['sheet-roll','sheet-adv','sheet-solo']})
        if len(rolls)==1:
            roll=rolls[0]
        elif len(rolls)==2:
            roll=rolls[0] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[1]
        else:
            log=False
        result=roll.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()

        tmplabel=mcontent[0].find('div',{'class': 'sheet-label'}).renderContents()
        if parsing=="Skill":
            tmplabel=re.sub(r'[^ A-Z0123456789-]','',str(tmplabel)).split(' ')
            label= [" ".join(tmplabel[:-1]),tmplabel[-1]] if len(tmplabel)>2 else tmplabel
        else:
            tmplabel=mcontent[0].find('div',{'class': 'sheet-label'}).renderContents()
            label=[re.sub("</*[a-zA-Z]+","",str(tmplabel).split(">")[2]),
                   re.sub("[^0-9-+]","",str(tmplabel).split(">")[4])]


        if  log and (label[0] not in ["D K","T T","INITIATIVE",""]) and ("HIT DICE" not in label[0]) and (label[0] not in attr_labels):
            df.loc[str(c), 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
            df.loc[str(c), 'Timestamp'] = when.decode('UTF-8')
            df.loc[str(c), 'Skill/Weapon/Spell'] = str(label[0])
            df.loc[str(c), 'Result'] = result.decode('UTF-8')
            df.loc[str(c), 'Raw'] = str(int(float(result))-int(label[1]))
            df.loc[str(c), 'Plus'] = str(label[1])
            df.loc[str(c), 'Final'] = 1
            df.loc[str(c), 'Type'] = parsing
            if len(rolls)==2:
                rollgrey=rolls[1] if rolls[1].find('span',{'class': 'sheet-grey'})!=None else rolls[0]
                resultgrey=rollgrey.find('span',{'class': lambda x: x and 'inlinerollresult' in x}).renderContents()
                df.loc[str(c)+".2", 'Player'] = re.sub(':','',plyr.decode('UTF-8'))
                df.loc[str(c)+".2", 'Timestamp'] = when.decode('UTF-8')
                df.loc[str(c)+".2", 'Skill/Weapon/Spell'] = str(label[0])
                df.loc[str(c)+".2", 'Result'] = resultgrey.decode('UTF-8')
                df.loc[str(c)+".2", 'Raw'] = str(int(float(resultgrey))-int(label[1]))
                df.loc[str(c)+".2", 'Plus'] = str(label[1])
                df.loc[str(c)+".2", 'Final'] = 0
                df.loc[str(c)+".2", 'Type'] = parsing
        
df["Skill/Weapon/Spell"]=df["Skill/Weapon/Spell"].str.strip()
df["Timestamp"]=pd.to_datetime(df["Timestamp"])
df["Player"]=df["Player"].apply(lambda x: players[x])
df['Day'] = df['Timestamp'].dt.strftime('%d.%m.%Y')
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")),"Type"]="Save"
df.loc[(df["Type"]=="Skill")&(df["Skill/Weapon/Spell"].isin(skill_labels)==False)&(df["Skill/Weapon/Spell"].str.contains("SAVE")==False),"Type"]="Other"
df=df[df["Player"].isin(plyrs)]
df.to_csv("rolls_"+name+".csv",index=False,sep=";")
df

Loading chat log...
Parsing messages...
1 / 850
501 / 850


,Player,Timestamp,Skill/Weapon/Spell,Result,Raw,Plus,Final,Type,Day
10,Smallfolk,2021-01-03 19:36:00,STEALTH,25,19,6,1.0,Skill,03.01.2021
11,Kallistē,2021-01-03 19:38:00,DECEPTION,20,17,3,1.0,Skill,03.01.2021
11.2,Kallistē,2021-01-03 19:38:00,DECEPTION,11,8,3,0.0,Skill,03.01.2021
12,Bethrynne,2021-01-03 19:50:00,HISTORY,8,2,6,1.0,Skill,03.01.2021
13,Smallfolk,2021-01-03 19:50:00,HISTORY,12,12,0,1.0,Skill,03.01.2021
...,...,...,...,...,...,...,...,...,...
840.2,Smallfolk,2021-02-08 00:38:00,Longbow,23,17,+6,0.0,Attack,08.02.2021
842,Smallfolk,2021-02-08 00:38:00,WISDOM SAVE,17,18,-1,1.0,Save,08.02.2021
843,Bethrynne,2021-02-08 00:39:00,ARCANA,10,4,6,1.0,Skill,08.02.2021
848,Bethrynne,2021-02-08 00:39:00,ARCANA,12,6,6,1.0,Skill,08.02.2021


In [16]:
df.Type.value_counts()

Skill     218
Attack    105
Save       22
Name: Type, dtype: int64